In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch

In [3]:
%cd '/content/drive/MyDrive/Lab/TabCSDI_plus'

/content/drive/MyDrive/Lab/TabCSDI_plus


In [4]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.1 MB/s eta 0:00:00


In [5]:
import pickle
import yaml
import os
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset

# 결과 확인

In [ ]:
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
          return super().find_class(module, name)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from src.main_model_table_ft import TabCSDI
from src.utils_table import train, evaluate_ft, evaluate_ft_all
from dataset_census.dataset_census_ft2 import get_dataloader

In [ ]:
# FT, MCAR
# bringing generation results
with open("./save/census_ft_fold5/MCARMCAR_20240310_072901/generated_outputs_nsample50.pk", 'rb') as f :
    if device == "cpu" : gnrt_ft_mcar20 = CPU_Unpickler(f).load()
    else : gnrt_ft_mcar20 = pickle.load(f)
with open("./save/census_ft_fold5/MCARMCAR_20240310_072901/full_generated_outputs_nsample50.pk", 'rb') as f :
    if device == "cpu" : gnrt_ft_mcar20 = CPU_Unpickler(f).load()
    else : full_ft_mcar20 = pickle.load(f)

samples = gnrt_ft_mcar20[0]
all_targets = gnrt_ft_mcar20[1]
all_evalpoints = gnrt_ft_mcar20[2]

fullsamples = full_ft_mcar20[0]
full_targets = full_ft_mcar20[1]
full_evalpoints = full_ft_mcar20[2]

# 모델 불러오기
modelfolder = "census_ft_fold5/MCARMCAR_20240310_072901" #model.pth가 있는 곳
testmissingratio = 0.2
config = "census_ft.yaml"
# device = "cuda"
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
exe_name = "census"

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

path = "config/" + config
with open(path, "r") as f:
    config = yaml.safe_load(f)
config["model"]["is_unconditional"] = 0
config["model"]["test_missing_ratio"] = testmissingratio
###
config["model"]["mecha"] = 'MCAR' ##
config["model"]["p_obs"] = 0.5 ##
###
model = TabCSDI(exe_name, config, device).to(device)

model.load_state_dict(torch.load("./save/" + modelfolder + "/model.pth",map_location=device))

with open("./data_census_ft/transformed_columns.pk", "rb") as f:
    cont_list, num_cate_list = pickle.load(f)
with open("./data_census_ft/encoder.pk", "rb") as f:
    encoder = pickle.load(f)
print(cont_list, num_cate_list)

model.eval()

# samples_median (데이터 합성 결과물), tokenizer recover (본래 데이터 형태로 되돌림)
samples_median = samples.median(dim=1)
samples_median = model.tokenizer.recover(samples_median.values, len(cont_list))

c_target = gnrt_ft_mcar20[1]
evalpoints = gnrt_ft_mcar20[2]
obspoints = gnrt_ft_mcar20[3]
evalpoints = evalpoints.squeeze()

c_target_ft = pd.DataFrame(np.array(c_target.cpu())) # real # minmax normalized
samples_ftmcar = pd.DataFrame(np.array(samples_median.cpu())) # synthetic
evalpoints_ftmcar = pd.DataFrame(np.array(evalpoints.cpu()))
target_imputed_ftmcar = c_target_ft.where(evalpoints_ftmcar==0,samples_ftmcar)

### full
# samples_median (데이터 합성 결과물), tokenizer recover (본래 데이터 형태로 되돌림)
fsamples_median = fullsamples.median(dim=1)
fsamples_median = model.tokenizer.recover(fsamples_median.values, len(cont_list))

fc_target = full_ft_mcar20[1]
fevalpoints = full_ft_mcar20[2]
fobspoints = full_ft_mcar20[3]
fevalpoints = fevalpoints.squeeze()

fc_target_ft = pd.DataFrame(np.array(fc_target.cpu())) # real # minmax normalized
fsamples_ftmcar = pd.DataFrame(np.array(fsamples_median.cpu())) # synthetic
fevalpoints_ftmcar = pd.DataFrame(np.array(fevalpoints.cpu()))
ftarget_imputed_ftmcar = fc_target_ft.where(fevalpoints_ftmcar==0,fsamples_ftmcar)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


In [ ]:
fc_target_ft

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.121622,0.330714,0.5625,0.00000,0.000265,0.000000,3.0,14.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.216216,0.157822,0.5625,0.05179,0.000265,0.454545,6.0,2.0,2.0,7.0,2.0,2.0,1.0,1.0,1.0
2,0.297297,0.000000,0.6250,0.00000,0.000265,0.404040,3.0,14.0,2.0,13.0,4.0,3.0,1.0,13.0,2.0
3,0.783784,0.059018,0.6250,0.00001,0.000000,0.090909,3.0,14.0,3.0,13.0,1.0,1.0,2.0,1.0,1.0
4,0.000000,0.071538,0.0000,0.00000,0.000265,0.242424,5.0,14.0,1.0,4.0,4.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.000000,0.115436,0.6250,0.00001,0.000265,0.000000,3.0,6.0,3.0,4.0,5.0,1.0,2.0,1.0,1.0
19996,0.297297,0.156011,0.5000,0.07689,0.000265,0.404040,3.0,16.0,2.0,7.0,2.0,2.0,1.0,1.0,2.0
19997,0.000000,0.143254,0.0000,0.00001,0.524529,0.505050,6.0,1.0,2.0,13.0,2.0,1.0,1.0,35.0,2.0
19998,0.405405,0.021001,1.0000,0.00001,0.415009,0.505050,4.0,14.0,1.0,4.0,1.0,2.0,2.0,1.0,2.0


In [ ]:
ftarget_imputed_ftmcar

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.112652e+06,-37985.984375,6275.697266,0.0,84903.765625,0.000000,5.0,14.0,2.0,11.0,3.0,5.0,1.0,32.0,2.0
1,7.507360e+05,-40345.308594,4906.801758,-4812080.5,86273.875000,74725.460938,6.0,12.0,2.0,11.0,1.0,2.0,1.0,26.0,1.0
2,7.504749e+05,0.000000,22615.605469,0.0,42938.972656,71876.390625,6.0,14.0,3.0,13.0,4.0,4.0,1.0,41.0,1.0
3,5.487181e+05,-26536.949219,58141.960938,-4621594.5,0.000000,43081.359375,5.0,14.0,3.0,13.0,3.0,2.0,1.0,17.0,1.0
4,0.000000e+00,-59679.527344,0.000000,0.0,58419.917969,32022.689453,5.0,14.0,8.0,11.0,2.0,3.0,1.0,26.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.000000e+00,-42915.523438,18484.214844,-5446551.5,63706.617188,0.000000,5.0,10.0,3.0,11.0,3.0,5.0,1.0,8.0,1.0
19996,3.954832e+05,-45011.492188,5915.923828,-4484783.0,46338.796875,49984.570312,6.0,15.0,8.0,8.0,1.0,2.0,1.0,17.0,1.0
19997,0.000000e+00,-37512.648438,0.000000,-4794043.5,95717.125000,47562.046875,8.0,15.0,2.0,13.0,3.0,5.0,1.0,35.0,1.0
19998,5.580072e+05,-57773.031250,-23012.605469,-5706241.5,61385.007812,54829.105469,7.0,14.0,6.0,8.0,2.0,2.0,1.0,30.0,1.0


In [ ]:
c_target_ft

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,0.188061,0.5625,0.00001,0.000265,0.40404,3.0,2.0,3.0,7.0,5.0,2.0,1.0,1.0,1.0
1,0.108108,0.000000,0.6250,0.00000,0.000265,0.40404,3.0,6.0,2.0,13.0,2.0,4.0,1.0,1.0,1.0
2,0.121622,0.089783,0.5625,0.00001,0.000000,0.40404,6.0,2.0,5.0,2.0,5.0,1.0,2.0,1.0,1.0
3,0.364865,0.000000,0.6250,0.00001,0.000265,0.50505,6.0,6.0,3.0,11.0,4.0,1.0,2.0,1.0,1.0
4,0.148649,0.153879,0.6875,0.00000,0.000265,0.40404,6.0,8.0,2.0,7.0,1.0,2.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.000000,0.115436,0.6250,0.00001,0.000265,0.00000,3.0,6.0,3.0,4.0,5.0,1.0,2.0,1.0,1.0
3996,0.297297,0.156011,0.5000,0.07689,0.000265,0.40404,3.0,16.0,2.0,7.0,2.0,2.0,1.0,1.0,2.0
3997,0.000000,0.143254,0.0000,0.00001,0.524529,0.50505,6.0,1.0,2.0,13.0,2.0,1.0,1.0,35.0,2.0
3998,0.405405,0.021001,1.0000,0.00001,0.415009,0.50505,4.0,14.0,1.0,4.0,1.0,2.0,2.0,1.0,2.0


In [ ]:
fevalpoints_ftmcar # 1이 missing이어야 하는데 반대로 된 듯

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
19996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
19997,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
19998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [ ]:
## 생성된 데이터 확인
path = '/content/drive/MyDrive/Lab/TabCSDI_plus/data_census_ft/missing_ratio-0.2_seed-1.pk'
with open(path, "rb") as f:
  observed_values, observed_masks, gt_masks, cont_cols = pickle.load(f)

In [ ]:
pd.DataFrame(observed_masks).sum()

0     16071
1     15963
2     16002
3     16071
4     15915
5     15964
6     16022
7     15923
8     15968
9     16014
10    16062
11    16071
12    16060
13    15896
14    15953
dtype: int64

In [ ]:
pd.DataFrame(gt_masks).sum()

0     16071.0
1     15963.0
2     16002.0
3     16071.0
4     15915.0
5     15964.0
6     16022.0
7     15923.0
8     15968.0
9     16014.0
10    16062.0
11    16071.0
12    16060.0
13    15896.0
14    15953.0
dtype: float64

In [ ]:
observed_values

array([[3.90000e+01, 7.75160e+04, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [5.00000e+01, 8.33110e+04, 1.30000e+01, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [3.80000e+01, 2.15646e+05, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [7.20000e+01, 2.68861e+05, 4.00000e+00, ..., 2.00000e+00,
        1.00000e+00, 1.00000e+00],
       [5.40000e+01, 3.43242e+05, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 2.00000e+00],
       [3.00000e+01, 4.60408e+05, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 2.00000e+00]])

In [ ]:
## 옛날거 - 생성된 데이터 확인
path = '/content/drive/MyDrive/Lab/TabCSDI/data_census_ft/missing_ratio-0.2_seed-1.pk'
with open(path, "rb") as f:
  observed_values, observed_masks, gt_masks, cont_cols = pickle.load(f)

In [ ]:
(pd.DataFrame(observed_values)==0).sum()

0         0
1         0
2         0
3     18357
4     19053
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
dtype: int64

In [ ]:
pd.DataFrame(observed_masks).sum()

0     20000
1     20000
2     20000
3     20000
4     20000
5     20000
6     20000
7     20000
8     20000
9     20000
10    20000
11    20000
12    20000
13    20000
14    20000
dtype: int64

In [ ]:
pd.DataFrame(gt_masks).sum()

0     16000
1     16000
2     16000
3     16000
4     16000
5     16000
6     16000
7     16000
8     16000
9     16000
10    16000
11    16000
12    16000
13    16000
14    16000
dtype: int64

# Run

In [ ]:
%run exe_census/exe_census_ft2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MCAR

Namespace(config='census_ft.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": true,
        "token_emb_dim": 8
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": true,
        "token_emb_dim": 8,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_ft_fold5/MCARMCAR_20240316_134433/
--------Normalized dataset loaded--------
Dataset size:200

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 188/188 [00:07<00:00, 24.32it/s, avg_epoch_loss=1.47e-5, epoch=99]


---------------Start testing---------------
[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


100%|██████████| 63/63 [58:35<00:00, 55.81s/it, rmse_total=115, batch_no=63]


RMSE: 115.31454467773438
ERR_CATE: [0.96296296 0.96171516 0.80837359 0.93343419 0.8449848  0.91693291
 0.3480916  0.99841772 0.2324159 ]
[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


100%|██████████| 313/313 [4:49:48<00:00, 55.56s/it, rmse_total=110, batch_no=313]


RMSE: 110.18406677246094
ERR_CATE: [0.98717949 0.9840569  0.99727183 0.71098846 0.46165566 0.95978621
 0.97715736 0.99951267 0.98937485]


In [ ]:
%run exe_census/exe_census_ft2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MAR

Namespace(config='census_ft.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.2, mecha1='MAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": true,
        "token_emb_dim": 8
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": true,
        "token_emb_dim": 8,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_ft_fold5/MARMCAR_20240317_041523/
--------Dataset created--------
Dataset size:20000 entries
-

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 188/188 [00:07<00:00, 24.68it/s, avg_epoch_loss=1.4e-5, epoch=99] 


---------------Start testing---------------
[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


 70%|██████▉   | 44/63 [39:18<16:57, 53.54s/it, rmse_total=548, batch_no=44]

In [ ]:
%run exe_census/exe_census_ft2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MAR --modelfolder census_ft_fold5/MARMCAR_20240317_041523

Namespace(config='census_ft.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='census_ft_fold5/MARMCAR_20240317_041523', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.2, mecha1='MAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": true,
        "token_emb_dim": 8
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": true,
        "token_emb_dim": 8,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_ft_fold5/MARMCAR_20240317_115217/
--------Normalized da

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


---------------Start testing---------------
[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


100%|██████████| 63/63 [55:08<00:00, 52.51s/it, rmse_total=535, batch_no=63]


RMSE: 535.2045288085938
ERR_CATE: [0.99404762 0.95508982 0.96078431 0.89423077 0.66362253 0.59404389
 0.50390016 0.99837925 0.3196347 ]
[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


100%|██████████| 313/313 [4:36:17<00:00, 52.96s/it, rmse_total=nan, batch_no=313]


RMSE: nan
ERR_CATE: [       nan 0.9994985         nan 0.94699211 0.92806285 0.99949887
 0.52245805 1.         0.87394744]


/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:925: RuntimeWarning: invalid value encountered in divide
  err_total / err_total_eval_nums,
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:935: RuntimeWarning: invalid value encountered in divide
  print("ERR_CATE:", err_total / err_total_eval_nums)


In [ ]:
%run exe_census/exe_census_ft2.py --seed 2 --nsample 50 --mecha MAR --testmissingratio 0.5 --p_obs 0.4 --mecha1 MCAR

Namespace(config='census_ft.yaml', device='cuda', seed=2, testmissingratio=0.5, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MAR', opt='logistic', p_obs=0.4, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": true,
        "token_emb_dim": 8
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": true,
        "token_emb_dim": 8,
        "test_missing_ratio": 0.5,
        "mecha": "MAR",
        "p_obs": 0.4
    }
}
model folder: ./save/census_ft_fold5/MCARMAR_20240317_174143/


ValueError: The function value at x=-50.0 is NaN; solver cannot continue.

In [ ]:
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MCAR

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_analog_fold5/MCARMCAR_20240319_092255/
--------Normalized dataset loaded--------
Dataset size:20000 entries
--------Normalized dataset l

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 188/188 [00:07<00:00, 24.99it/s, avg_epoch_loss=8.32e-6, epoch=99]


---------------Start testing---------------


 30%|███       | 19/63 [16:37<38:05, 51.95s/it, rmse_total=1.3, batch_no=19]

In [ ]:
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MCAR --modelfolder census_analog_fold5/MCARMCAR_20240319_092255

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='census_analog_fold5/MCARMCAR_20240319_092255', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_analog_fold5/MCARMCAR_20240319_115432/
--------Normalized dataset loaded--------
Dataset siz

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


---------------Start testing---------------


100%|██████████| 63/63 [54:07<00:00, 51.54s/it, rmse_total=1.33, batch_no=63]


RMSE: 1.3297713994979858
ERR_CATE: [1.         0.959375   0.9306184  1.         0.77794793 0.84057971
 0.60816944 0.99683544 0.97706422]


100%|██████████| 313/313 [4:24:52<00:00, 50.77s/it, rmse_total=0.728, batch_no=313]


RMSE: 0.7280893325805664
ERR_CATE: [0.00099083 0.7640621  0.88875124 0.00326797 0.82781457 0.97395833
 0.84696438 0.41569201 0.08574253]


In [ ]:
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MCAR --modelfolder census_analog_fold5/MCARMCAR_20240319_092255

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='census_analog_fold5/MCARMCAR_20240319_092255', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_analog_fold5/MCARMCAR_20240320_112245/
--------Dataset created--------
Dataset size:20000 en

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


---------------Start testing---------------


100%|██████████| 313/313 [00:00<00:00, 443.37it/s]


In [ ]:
# MAR, MCAR
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MCAR --mecha1 MAR --missingratio1 0.5 --p_obs1 0.4

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.2, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MCAR', opt='logistic', p_obs=0.2, missingratio1=0.5, mecha1='MAR', p_obs1=0.4)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.2,
        "mecha": "MCAR",
        "p_obs": 0.2
    }
}
model folder: ./save/census_analog_fold5/MARMCAR_20240320_123318/
--------Dataset created--------
Dataset size:20000 entries
--------------Dataset has not been norma

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 188/188 [00:09<00:00, 20.19it/s, avg_epoch_loss=7.38e-6, epoch=99]


---------------Start testing---------------


100%|██████████| 63/63 [1:05:04<00:00, 61.98s/it, rmse_total=0.812, batch_no=63]


RMSE: 0.8120233416557312
ERR_CATE: [0.93386243 0.96640827 1.         0.99761905 0.93617021 0.88627451
 0.57160648 0.99745547 0.68130489]


100%|██████████| 313/313 [5:22:46<00:00, 61.87s/it, rmse_total=nan, batch_no=313]


RMSE: nan
ERR_CATE: [       nan 0.96757028 0.99909228        nan 0.92762231        nan
        nan 0.99868421        nan]


/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:1078: RuntimeWarning: invalid value encountered in divide
  err_total / err_total_eval_nums,
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:1088: RuntimeWarning: invalid value encountered in divide
  print("ERR_CATE:", err_total / err_total_eval_nums)


In [ ]:
# MCAR MAR
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MAR --mecha1 MCAR --testmissingratio 0.5 --p_obs 0.4

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.5, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MAR', opt='logistic', p_obs=0.4, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.5,
        "mecha": "MAR",
        "p_obs": 0.4
    }
}
model folder: ./save/census_analog_fold5/MCARMAR_20240321_080014/
--------Dataset created--------
Dataset size:20000 entries
--------------Dataset has not been normal

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 188/188 [00:14<00:00, 13.14it/s, avg_epoch_loss=1.76e-5, epoch=99]


---------------Start testing---------------


100%|██████████| 63/63 [54:04<00:00, 51.50s/it, rmse_total=nan, batch_no=63]
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:338: RuntimeWarning: invalid value encountered in divide
  err_total / err_total_eval_nums,
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:348: RuntimeWarning: invalid value encountered in divide
  print("ERR_CATE:", err_total / err_total_eval_nums)


RMSE: nan
ERR_CATE: [0.9025894         nan 0.99938688        nan        nan 0.84944238
 1.         0.99813317 0.82009627]


 27%|██▋       | 83/313 [1:10:50<3:14:22, 50.71s/it, rmse_total=0.6, batch_no=83]

In [ ]:
# MCAR MAR
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MAR --mecha1 MCAR --testmissingratio 0.5 --p_obs 0.4 --modelfolder census_analog_fold5/MCARMAR_20240321_080014

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.5, nfold=5, unconditional=0, modelfolder='census_analog_fold5/MCARMAR_20240321_080014', nsample=50, mecha='MAR', opt='logistic', p_obs=0.4, missingratio1=0.2, mecha1='MCAR', p_obs1=0.2)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.5,
        "mecha": "MAR",
        "p_obs": 0.4
    }
}
model folder: ./save/census_analog_fold5/MCARMAR_20240321_105225/
--------Normalized dataset loaded--------
Dataset size:20

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


---------------Start testing---------------


100%|██████████| 313/313 [4:22:22<00:00, 50.30s/it, rmse_total=0.599, batch_no=313]


RMSE: 0.5993825197219849
ERR_CATE: [0.90116423 0.99058285 0.99900892 0.90573152 0.98700025 0.85119048
 1.         0.99683236 0.79960465]


In [ ]:
# MAR MAR
%run exe_census/exe_census_analog2.py --seed 2 --nsample 50 --mecha MAR --mecha1 MAR --testmissingratio 0.5 --p_obs 0.4 --missingratio1 0.5 --p_obs1 0.4

Namespace(config='census_onehot_analog.yaml', device='cuda', seed=2, testmissingratio=0.5, nfold=5, unconditional=0, modelfolder='', nsample=50, mecha='MAR', opt='logistic', p_obs=0.4, missingratio1=0.5, mecha1='MAR', p_obs1=0.4)
{
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "lr": 0.0005
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": false
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 8,
        "target_strategy": "random",
        "mixed": false,
        "test_missing_ratio": 0.5,
        "mecha": "MAR",
        "p_obs": 0.4
    }
}
model folder: ./save/census_analog_fold5/MARMAR_20240321_162138/
--------Dataset created--------
Dataset size:20000 entries
--------------Dataset has not been normaliz

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 188/188 [00:14<00:00, 13.08it/s, avg_epoch_loss=1.92e-5, epoch=99]


---------------Start testing---------------


100%|██████████| 63/63 [53:10<00:00, 50.64s/it, rmse_total=nan, batch_no=63]
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:338: RuntimeWarning: invalid value encountered in divide
  err_total / err_total_eval_nums,
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:348: RuntimeWarning: invalid value encountered in divide
  print("ERR_CATE:", err_total / err_total_eval_nums)


RMSE: nan
ERR_CATE: [0.89481409        nan 0.99442897        nan        nan 0.77683897
 1.         0.99587629 0.90904548]


100%|██████████| 313/313 [4:32:17<00:00, 52.20s/it, rmse_total=nan, batch_no=313]


RMSE: nan
ERR_CATE: [       nan 0.99457831 0.99283913        nan 0.98711496        nan
        nan 0.99736842        nan]


/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:1078: RuntimeWarning: invalid value encountered in divide
  err_total / err_total_eval_nums,
/content/drive/MyDrive/Lab/TabCSDI_plus/src/utils_table.py:1088: RuntimeWarning: invalid value encountered in divide
  print("ERR_CATE:", err_total / err_total_eval_nums)


# 결과 확인

## 함수 세팅

In [7]:
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
          return super().find_class(module, name)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
from src.main_model_table_ft import TabCSDI
from src.utils_table import train, evaluate_ft, evaluate_ft_all
from dataset_census.dataset_census_ft2 import get_dataloader

In [10]:
# bringing generation results
def open_results_ft(path_model, path_1, path_2, testmissingratio, p_obs, mecha):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with open(f"./save/{path_1}/generated_outputs_nsample50.pk", 'rb') as f :
        if device == "cpu" : gnrt_ft_mcar20 = CPU_Unpickler(f).load()
        else : gnrt_ft_mcar20 = pickle.load(f)
    with open(f"./save/{path_2}/full_generated_outputs_nsample50.pk", 'rb') as f :
        if device == "cpu" : gnrt_ft_mcar20 = CPU_Unpickler(f).load()
        else : full_ft_mcar20 = pickle.load(f)

    samples = gnrt_ft_mcar20[0]
    all_targets = gnrt_ft_mcar20[1]
    all_evalpoints = gnrt_ft_mcar20[2]

    fullsamples = full_ft_mcar20[0]
    full_targets = full_ft_mcar20[1]
    full_evalpoints = full_ft_mcar20[2]

    # 모델 불러오기
    modelfolder = path_model #model.pth가 있는 곳
    # testmissingratio = 0.2
    config = "census_ft.yaml"
    exe_name = "census"

    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

    path = "config/" + config
    with open(path, "r") as f:
        config = yaml.safe_load(f)
    config["model"]["is_unconditional"] = 0
    config["model"]["test_missing_ratio"] = testmissingratio
    ###
    config["model"]["mecha"] = mecha ##
    config["model"]["p_obs"] = p_obs ##
    ###
    model = TabCSDI(exe_name, config, device).to(device)

    model.load_state_dict(torch.load("./save/" + modelfolder + "/model.pth",map_location=device))

    with open("./data_census_ft/transformed_columns.pk", "rb") as f:
        cont_list, num_cate_list = pickle.load(f)
    with open("./data_census_ft/encoder.pk", "rb") as f:
        encoder = pickle.load(f)
    print(cont_list, num_cate_list)

    model.eval()

    # samples_median (데이터 합성 결과물), tokenizer recover (본래 데이터 형태로 되돌림)
    samples_median = samples.median(dim=1)
    samples_median = model.tokenizer.recover(samples_median.values, len(cont_list)) # input: [4000,120,1]

    c_target = gnrt_ft_mcar20[1]
    evalpoints = gnrt_ft_mcar20[2]
    obspoints = gnrt_ft_mcar20[3]
    evalpoints = evalpoints.squeeze()

    c_target_ft = pd.DataFrame(np.array(c_target.cpu())) # real # minmax normalized
    samples_ftmcar = pd.DataFrame(np.array(samples_median.cpu())) # synthetic
    evalpoints_ftmcar = pd.DataFrame(np.array(evalpoints.cpu()))
    target_imputed_ftmcar = c_target_ft.where(evalpoints_ftmcar==0,samples_ftmcar)

    ### full
    # samples_median (데이터 합성 결과물), tokenizer recover (본래 데이터 형태로 되돌림)
    fsamples_median = fullsamples.median(dim=1)
    fsamples_median = model.tokenizer.recover(fsamples_median.values, len(cont_list))

    fc_target = full_ft_mcar20[1]
    fevalpoints = full_ft_mcar20[2]
    fobspoints = full_ft_mcar20[3]
    fevalpoints = fevalpoints.squeeze()

    fc_target_ft = pd.DataFrame(np.array(fc_target.cpu())) # real # minmax normalized
    fsamples_ftmcar = pd.DataFrame(np.array(fsamples_median.cpu())) # synthetic
    fevalpoints_ftmcar = pd.DataFrame(np.array(fevalpoints.cpu()))
    ftarget_imputed_ftmcar = fc_target_ft.where(fevalpoints_ftmcar==0,fsamples_ftmcar)
    return c_target_ft, samples_ftmcar, evalpoints_ftmcar, target_imputed_ftmcar, fc_target_ft, fsamples_ftmcar, fevalpoints_ftmcar, ftarget_imputed_ftmcar

In [23]:
# Analog, MCAR
# bringing generation results
def open_results_an(path_model, path_1, path_2, testmissingratio, p_obs, mecha): # for test set
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with open(f"./save/{path_1}/generated_outputs_nsample50.pk", 'rb') as f :
        if device == "cpu" : gnrt_an_mcar20 = CPU_Unpickler(f).load()
        else : gnrt_an_mcar20 = pickle.load(f)
    with open(f"./save/{path_2}/full_generated_outputs_nsample50.pk", 'rb') as f :
        if device == "cpu" : gnrt_an_mcar20 = CPU_Unpickler(f).load()
        else : full_an_mcar20 = pickle.load(f)

    modelfolder = path_model #
    config = "census_onehot_analog.yaml"
    exe_name = "census"

    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

    path = "config/" + config
    with open(path, "r") as f:
        config = yaml.safe_load(f)
    config["model"]["is_unconditional"] = 0
    config["model"]["test_missing_ratio"] = testmissingratio
    ###
    config["model"]["mecha"] = mecha ##
    config["model"]["p_obs"] = p_obs ##
    ###

    model = TabCSDI(config, device).to(device)

    model.load_state_dict(torch.load("./save/" + modelfolder + "/model.pth",map_location=device))
    model.eval()

    samples = gnrt_an_mcar20[0] #
    samples_median = samples.median(dim=1)
    c_target = gnrt_an_mcar20[1] #
    evalpoints = gnrt_an_mcar20[2] #
    obspoints = gnrt_an_mcar20[3] #
    evalpoints = evalpoints.squeeze()

    fsamples = full_an_mcar20[0] #
    fsamples_median = fsamples.median(dim=1)
    fc_target = full_an_mcar20[1] #
    fevalpoints = full_an_mcar20[2] #
    fobspoints = full_an_mcar20[3] #
    fevalpoints = fevalpoints.squeeze()

    ### real data 불러오기
    with open("./save/census_analog_fold5/MCARMCAR_20240319_115432_/full_real_data.pk", 'rb') as f :
        real1 = CPU_Unpickler(f).load()
    ###

    if exe_name == "census":
        with open("./data_census_analog/transform.pk", "rb") as f:
            _, cont_cols, saved_cat_dict = pickle.load(f)
        print(cont_cols, saved_cat_dict)

    # Threashold
    for i in range(samples_median.values.size(dim=1)):
      if i in cont_cols:
        continue
      index1 = samples_median.values[:, i, :] >= 0
      samples_median.values[:, i, :][index1] = 1
      index2 = samples_median.values[:, i, :] < 0
      samples_median.values[:, i, :][index2] = -1

    # Threashold
    for i in range(fsamples_median.values.size(dim=1)):
      if i in cont_cols:
        continue
      index1 = fsamples_median.values[:, i, :] >= 0
      fsamples_median.values[:, i, :][index1] = 1
      index2 = fsamples_median.values[:, i, :] < 0
      fsamples_median.values[:, i, :][index2] = -1

    samples_median_df = pd.DataFrame(samples_median.values.squeeze().cpu())
    evalpoints_anmcar = pd.DataFrame(np.array(evalpoints.cpu())) #
    samples_median_anmcar = samples_median_df * evalpoints_anmcar #
    c_target_an = pd.DataFrame(c_target.squeeze().cpu())
    target_imputed_anmcar = c_target_an.where(evalpoints_anmcar==0,samples_median_anmcar) #

    fsamples_median_df = pd.DataFrame(fsamples_median.values.squeeze().cpu())
    fevalpoints_anmcar = pd.DataFrame(np.array(fevalpoints.cpu())) #
    fsamples_median_anmcar = fsamples_median_df * fevalpoints_anmcar #
    fc_target_an = pd.DataFrame(fc_target.squeeze().cpu())
    ftarget_imputed_anmcar = fc_target_an.where(fevalpoints_anmcar==0,fsamples_median_anmcar) #
    ###
    freal_an = pd.DataFrame(real1[0].squeeze().cpu())

    return c_target_an, samples_median_anmcar, evalpoints_anmcar, target_imputed_anmcar, fc_target_an, fsamples_median_anmcar, fevalpoints_anmcar, ftarget_imputed_anmcar, freal_an

In [24]:
# Analog,2 (저장값에 real을 포함한 이후 돌린 것들에 쓸 함수. 다시 맨 뒤로 보내놓긴 함...)
# bringing generation results
def open_results_an2(path_model, path_1, path_2, testmissingratio, p_obs, mecha): # for test set
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with open(f"./save/{path_1}/generated_outputs_nsample50.pk", 'rb') as f :
        if device == "cpu" : gnrt_an_mcar20 = CPU_Unpickler(f).load()
        else : gnrt_an_mcar20 = pickle.load(f)
    with open(f"./save/{path_2}/full_generated_outputs_nsample50.pk", 'rb') as f :
        if device == "cpu" : gnrt_an_mcar20 = CPU_Unpickler(f).load()
        else : full_an_mcar20 = pickle.load(f)

    modelfolder = path_model #
    config = "census_onehot_analog.yaml"
    exe_name = "census"

    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

    path = "config/" + config
    with open(path, "r") as f:
        config = yaml.safe_load(f)
    config["model"]["is_unconditional"] = 0
    config["model"]["test_missing_ratio"] = testmissingratio
    ###
    config["model"]["mecha"] = mecha ##
    config["model"]["p_obs"] = p_obs ##
    ###

    model = TabCSDI(config, device).to(device)

    model.load_state_dict(torch.load("./save/" + modelfolder + "/model.pth",map_location=device))
    model.eval()

    samples = gnrt_an_mcar20[0] #
    samples_median = samples.median(dim=1)
    c_target = gnrt_an_mcar20[1] #
    evalpoints = gnrt_an_mcar20[2] #
    obspoints = gnrt_an_mcar20[3] #
    evalpoints = evalpoints.squeeze()

    fsamples = full_an_mcar20[0] #
    fsamples_median = fsamples.median(dim=1)
    fc_target = full_an_mcar20[2] #
    fevalpoints = full_an_mcar20[3] #
    fobspoints = full_an_mcar20[4] #
    fevalpoints = fevalpoints.squeeze()
    freal = full_an_mcar20[1] ###

    if exe_name == "census":
        with open("./data_census_analog/transform.pk", "rb") as f:
            _, cont_cols, saved_cat_dict = pickle.load(f)
        print(cont_cols, saved_cat_dict)

    # Threashold
    for i in range(samples_median.values.size(dim=1)):
      if i in cont_cols:
        continue
      index1 = samples_median.values[:, i, :] >= 0
      samples_median.values[:, i, :][index1] = 1
      index2 = samples_median.values[:, i, :] < 0
      samples_median.values[:, i, :][index2] = -1

    # Threashold
    for i in range(fsamples_median.values.size(dim=1)):
      if i in cont_cols:
        continue
      index1 = fsamples_median.values[:, i, :] >= 0
      fsamples_median.values[:, i, :][index1] = 1
      index2 = fsamples_median.values[:, i, :] < 0
      fsamples_median.values[:, i, :][index2] = -1

    samples_median_df = pd.DataFrame(samples_median.values.squeeze().cpu())
    evalpoints_anmcar = pd.DataFrame(np.array(evalpoints.cpu())) #
    samples_median_anmcar = samples_median_df * evalpoints_anmcar #
    c_target_an = pd.DataFrame(c_target.squeeze().cpu())
    target_imputed_anmcar = c_target_an.where(evalpoints_anmcar==0,samples_median_anmcar) #

    fsamples_median_df = pd.DataFrame(fsamples_median.values.squeeze().cpu())
    fevalpoints_anmcar = pd.DataFrame(np.array(fevalpoints.cpu())) #
    fsamples_median_anmcar = fsamples_median_df * fevalpoints_anmcar #
    fc_target_an = pd.DataFrame(fc_target.squeeze().cpu())
    freal_an = pd.DataFrame(freal.squeeze().cpu()) ###
    ftarget_imputed_anmcar = fc_target_an.where(fevalpoints_anmcar==0,fsamples_median_anmcar) #

    return c_target_an, samples_median_anmcar, evalpoints_anmcar, target_imputed_anmcar, fc_target_an, fsamples_median_anmcar, fevalpoints_anmcar, ftarget_imputed_anmcar, freal_an

In [13]:
# Analog 결과 계산 함수
def analog_results(evalpoints, target, imputed, result_nums, fevalpoints, ftarget, fimputed, fresult_nums, freal):
    an_cont_list = [0, 5, 11, 28, 29, 30]
    evalpoints_cont = evalpoints.iloc[:,an_cont_list]
    diff = (target-imputed).iloc[:,an_cont_list]
    diff = diff.where(evalpoints_cont==1)
    rmse = np.sqrt((diff ** 2).mean())
    rmse_avg = result_nums[0]
    # error = result_an_mcarmcar[1]
    # error_avg = error.mean()

    with open("./data_census_analog/transform.pk", "rb") as f:
        _, cont_cols, saved_cat_dict = pickle.load(f)

    err_total = np.zeros([len(saved_cat_dict)])
    err_total_eval_nums = np.zeros([len(saved_cat_dict)])
    for i in range(len(saved_cat_dict)):
        cate_cols = saved_cat_dict[str(i)]
        matched_nums = (
            (
                (
                    imputed.values[:, cate_cols] # samples -> imputed
                    == target.values[:, cate_cols]
                )
                * evalpoints.values[:, cate_cols]
            )
            .all(1)
            .sum()
        )
        eval_nums = evalpoints.values[:, cate_cols].sum() / len(
            cate_cols
        )
        err_total[i] += eval_nums - matched_nums
        err_total_eval_nums[i] += eval_nums

    error = err_total / err_total_eval_nums
    error_avg = error.mean()

    ###
    fevalpoints_cont = fevalpoints.iloc[:,an_cont_list]
    fdiff = (freal-fimputed).iloc[:,an_cont_list]
    fdiff = fdiff.where(fevalpoints_cont==1)
    frmse = np.sqrt((fdiff ** 2).mean())
    frmse_avg = fresult_nums[0]

    ferr_total = np.zeros([len(saved_cat_dict)])
    ferr_total_eval_nums = np.zeros([len(saved_cat_dict)])
    for i in range(len(saved_cat_dict)):
        cate_cols = saved_cat_dict[str(i)]
        fmatched_nums = (
            (
                (
                    fimputed.values[:, cate_cols] ###
                    == freal.values[:, cate_cols]
                )
                * fevalpoints.values[:, cate_cols]
            )
            .all(1)
            .sum()
        )
        feval_nums = fevalpoints.values[:, cate_cols].sum() / len(
            cate_cols
        )
        ferr_total[i] += feval_nums - fmatched_nums
        ferr_total_eval_nums[i] += feval_nums

    ferror = ferr_total / ferr_total_eval_nums
    ferror_avg = ferror.mean()

    return rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg

In [14]:
# Analog 결과 계산 함수 - 2 (real로 비교)
def analog_results2(evalpoints, target, imputed, result_nums, fevalpoints, ftarget, fimputed, fresult_nums, freal):
    an_cont_list = [0, 5, 11, 28, 29, 30]
    evalpoints_cont = evalpoints.iloc[:,an_cont_list]
    diff = (target-imputed).iloc[:,an_cont_list]
    diff = diff.where(evalpoints_cont==1)
    rmse = np.sqrt((diff ** 2).mean())
    rmse_avg = result_nums[0]
    # error = result_an_mcarmcar[1]
    # error_avg = error.mean()

    with open("./data_census_analog/transform.pk", "rb") as f:
        _, cont_cols, saved_cat_dict = pickle.load(f)

    err_total = np.zeros([len(saved_cat_dict)])
    err_total_eval_nums = np.zeros([len(saved_cat_dict)])
    for i in range(len(saved_cat_dict)):
        cate_cols = saved_cat_dict[str(i)]
        matched_nums = (
            (
                (
                    imputed.values[:, cate_cols] # samples -> imputed
                    == target.values[:, cate_cols]
                )
                * evalpoints.values[:, cate_cols]
            )
            .all(1)
            .sum()
        )
        eval_nums = evalpoints.values[:, cate_cols].sum() / len(
            cate_cols
        )
        err_total[i] += eval_nums - matched_nums
        err_total_eval_nums[i] += eval_nums

    error = err_total / err_total_eval_nums
    error_avg = error.mean()

    fevalpoints_cont = fevalpoints.iloc[:,an_cont_list]
    fdiff = (freal-fimputed).iloc[:,an_cont_list]
    fdiff = fdiff.where(fevalpoints_cont==1)
    frmse = np.sqrt((fdiff ** 2).mean())
    # frmse_avg = fresult_nums[0]
    frmse_avg = frmse.mean()

    ferr_total = np.zeros([len(saved_cat_dict)])
    ferr_total_eval_nums = np.zeros([len(saved_cat_dict)])
    for i in range(len(saved_cat_dict)):
        cate_cols = saved_cat_dict[str(i)]
        fmatched_nums = (
            (
                (
                    fimputed.values[:, cate_cols] ###
                    == freal.values[:, cate_cols]
                )
                * fevalpoints.values[:, cate_cols]
            )
            .all(1)
            .sum()
        )
        feval_nums = fevalpoints.values[:, cate_cols].sum() / len(
            cate_cols
        )
        ferr_total[i] += feval_nums - fmatched_nums
        ferr_total_eval_nums[i] += feval_nums

    ferror = ferr_total / ferr_total_eval_nums
    ferror_avg = ferror.mean()

    return rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg

In [15]:
path_model = "census_ft_fold5/MCARMCAR_20240316_134433"
path_1 = 'census_ft_fold5/MCARMCAR_20240316_134433'
path_2 = 'census_ft_fold5/MCARMCAR_20240316_134433'

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, a, b = open_results(path_model, path_1, path_2)

NameError: name 'open_results' is not defined

## 대충 확인

In [ ]:
pd.DataFrame(np.array(a.cpu()))
### -exp2 파일에서 똑같이 이 결과 (median 취하기 전에 recover한 상태) 확인해보기.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-106093.851562,3163.618164,29088.097656,-107086.000000,40166.761719,-220.284454,7.0,3.0,4.0,4.0,4.0,2.0,1.0,10.0,2.0
1,-392350.718750,-85601.781250,117184.125000,6793.384277,-190678.156250,-88399.062500,3.0,9.0,3.0,12.0,2.0,4.0,1.0,24.0,2.0
2,274.196289,28037.537109,-66.864326,-44.106377,37161.230469,155039.250000,4.0,7.0,3.0,5.0,1.0,2.0,1.0,32.0,2.0
3,47393.226562,39938.640625,-25416.197266,-8110.726562,119405.429688,136850.906250,7.0,7.0,4.0,10.0,5.0,2.0,1.0,2.0,2.0
4,-1321.298340,139627.078125,-678.249817,-1080.042969,-16547.841797,2127.953125,7.0,6.0,5.0,14.0,1.0,4.0,2.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,64735.242188,100505.890625,5395.922852,588.414246,147919.765625,-11560.968750,6.0,5.0,5.0,3.0,3.0,5.0,2.0,2.0,1.0
3996,-295677.468750,-126967.664062,-78.498924,-79659.101562,-110729.562500,-36750.679688,7.0,15.0,3.0,1.0,3.0,2.0,1.0,2.0,2.0
3997,-1491.302124,-75061.593750,236909.562500,7395.829590,-51154.023438,-65114.640625,7.0,15.0,5.0,8.0,5.0,1.0,1.0,4.0,2.0
3998,26733.523438,-88343.117188,125830.164062,-143910.781250,-337.991150,-143869.687500,7.0,14.0,2.0,2.0,4.0,4.0,2.0,14.0,1.0


In [ ]:
pd.DataFrame(np.array(b.cpu()))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,389832.687500,-1856.122559,305.975250,12699.982422,1520.339844,8.160149,3.0,13.0,8.0,5.0,4.0,5.0,1.0,21.0,1.0
1,82773.867188,-84707.890625,-5932.921875,-6.024277,36840.812500,9845.759766,1.0,9.0,4.0,1.0,1.0,5.0,1.0,31.0,2.0
2,-11490.796875,5398.387207,7505.057129,-13.948181,2496.718750,51563.875000,2.0,6.0,4.0,5.0,2.0,5.0,2.0,32.0,2.0
3,-313876.531250,-6524.156250,-296104.937500,125.397797,69358.468750,159123.421875,8.0,6.0,8.0,10.0,4.0,5.0,2.0,20.0,1.0
4,-7404.340820,-75535.875000,35967.820312,2078.875000,160.849548,-24903.570312,8.0,4.0,4.0,14.0,3.0,5.0,2.0,32.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,346474.468750,76242.414062,-131.613647,-753.570801,-67898.007812,181306.187500,7.0,15.0,2.0,7.0,3.0,2.0,1.0,10.0,1.0
3996,283784.625000,-27380.781250,1395.523438,-55894.265625,59201.687500,-21107.363281,4.0,11.0,4.0,7.0,4.0,4.0,1.0,22.0,2.0
3997,-33.192871,-149652.437500,-53692.921875,-13.091355,-99558.640625,-24071.296875,7.0,14.0,4.0,9.0,4.0,3.0,2.0,32.0,1.0
3998,90997.710938,43530.929688,84031.343750,33610.238281,125.044800,127035.562500,2.0,16.0,7.0,14.0,3.0,4.0,2.0,19.0,1.0


In [ ]:
evalpoints

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3997,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
fevalpoints

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
19998,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
target

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.054054,0.176333,0.0625,0.00001,0.00023,0.323232,3.0,15.0,1.0,5.0,4.0,1.0,1.0,1.0,1.0
1,0.094595,0.113452,0.8125,0.00000,0.00023,0.404040,3.0,1.0,7.0,4.0,4.0,1.0,2.0,1.0,1.0
2,0.486487,0.092898,0.5625,0.00001,0.00023,0.484848,3.0,2.0,2.0,2.0,2.0,5.0,1.0,24.0,1.0
3,0.121622,0.197108,0.8125,0.00001,0.00023,0.404040,3.0,1.0,1.0,7.0,1.0,5.0,1.0,5.0,1.0
4,0.000000,0.155764,0.9375,0.00000,0.00000,0.000000,3.0,1.0,2.0,10.0,4.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.081081,0.308550,0.0000,0.00001,0.00023,0.080808,3.0,1.0,7.0,11.0,4.0,1.0,1.0,1.0,1.0
3996,0.351351,0.204480,0.0000,0.00001,0.00023,0.404040,3.0,1.0,3.0,10.0,1.0,1.0,2.0,1.0,1.0
3997,0.418919,0.122667,0.8125,0.00000,0.00023,0.606061,3.0,1.0,2.0,10.0,2.0,1.0,2.0,1.0,2.0
3998,0.500000,0.067009,0.3125,0.00001,0.00000,0.363636,3.0,5.0,3.0,10.0,1.0,1.0,2.0,10.0,1.0


In [ ]:
samples

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-52897.511719,10.246449,-58.281319,-41992.312500,-2739.196777,-66.830185,7.0,15.0,4.0,14.0,4.0,4.0,1.0,10.0,2.0
1,-16602.609375,23505.595703,39350.445312,-16.564270,198.325928,25667.800781,4.0,9.0,3.0,5.0,4.0,4.0,1.0,14.0,2.0
2,326.537903,-115.391685,86.556427,-10.932308,9782.195312,43754.140625,4.0,9.0,8.0,5.0,4.0,2.0,1.0,20.0,2.0
3,-105528.843750,17498.257812,35487.253906,-20.867718,-16092.041992,75170.945312,5.0,7.0,4.0,8.0,4.0,3.0,1.0,40.0,2.0
4,14.690653,-12768.355469,174.467667,-98.993721,94.403503,-713.452148,8.0,9.0,4.0,14.0,4.0,4.0,1.0,22.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-36446.558594,8525.626953,175.328995,58.930599,27095.888672,73795.734375,5.0,3.0,2.0,10.0,6.0,1.0,1.0,1.0,2.0
3996,-10123.209961,-20905.380859,14.525583,-31441.687500,18698.873047,77321.812500,4.0,7.0,4.0,5.0,4.0,1.0,1.0,22.0,2.0
3997,-32.704956,-26412.300781,41804.921875,-54.857208,7572.657227,-17.246094,5.0,13.0,4.0,2.0,6.0,5.0,1.0,3.0,1.0
3998,4773.948242,-11845.371094,26332.966797,-41394.625000,-2.129350,64939.828125,4.0,8.0,4.0,13.0,5.0,1.0,1.0,41.0,2.0


In [ ]:
imputed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.054054,10.246449,-58.281319,0.000010,0.00023,-66.830185,7.0,15.0,1.0,5.0,4.0,1.0,1.0,1.0,1.0
1,0.094595,0.113452,0.812500,0.000000,0.00023,0.404040,3.0,1.0,7.0,4.0,4.0,1.0,2.0,1.0,1.0
2,326.537903,-115.391685,86.556427,-10.932308,0.00023,0.484848,3.0,2.0,2.0,2.0,2.0,5.0,1.0,24.0,1.0
3,0.121622,0.197108,0.812500,-20.867718,0.00023,0.404040,3.0,1.0,1.0,8.0,1.0,5.0,1.0,40.0,1.0
4,0.000000,0.155764,174.467667,0.000000,0.00000,0.000000,8.0,1.0,2.0,10.0,4.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.081081,0.308550,0.000000,58.930599,0.00023,0.080808,3.0,1.0,7.0,10.0,6.0,1.0,1.0,1.0,1.0
3996,0.351351,0.204480,0.000000,0.000010,0.00023,0.404040,3.0,1.0,3.0,10.0,1.0,1.0,2.0,1.0,1.0
3997,-32.704956,0.122667,0.812500,0.000000,0.00023,0.606061,5.0,1.0,2.0,10.0,2.0,1.0,2.0,1.0,1.0
3998,0.500000,0.067009,0.312500,0.000010,0.00000,0.363636,3.0,5.0,3.0,10.0,1.0,1.0,2.0,10.0,1.0


In [ ]:
## 생성된 데이터 확인
path = '/content/drive/MyDrive/Lab/TabCSDI_plus/data_census_ft/MCAR-MCAR_seed-2.pk'
with open(path, "rb") as f:
  observed_values, observed_masks, gt_masks, cont_cols = pickle.load(f)

In [ ]:
pd.DataFrame(observed_masks).sum()

0     16071
1     15963
2     16002
3     16071
4     15915
5     15964
6     16022
7     15923
8     15968
9     16014
10    16062
11    16071
12    16060
13    15896
14    15953
dtype: int64

In [ ]:
pd.DataFrame(gt_masks).sum() # observed masks보다 작은거 맞음. 0(missing)이 더 많아지니까!

0     12818.0
1     12789.0
2     12779.0
3     12950.0
4     12782.0
5     12721.0
6     12760.0
7     12697.0
8     12819.0
9     12744.0
10    12892.0
11    12824.0
12    12878.0
13    12776.0
14    12814.0
dtype: float64

In [ ]:
observed_values

array([[3.90000e+01, 7.75160e+04, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [5.00000e+01, 8.33110e+04, 1.30000e+01, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [3.80000e+01, 2.15646e+05, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [7.20000e+01, 2.68861e+05, 4.00000e+00, ..., 2.00000e+00,
        1.00000e+00, 1.00000e+00],
       [5.40000e+01, 3.43242e+05, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 2.00000e+00],
       [3.00000e+01, 4.60408e+05, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 2.00000e+00]])

In [ ]:
pd.DataFrame(observed_masks)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,True,True,False,False,True,True,True,True,True,True,True,True,False,False,True
1,True,True,True,False,True,True,True,True,True,True,True,True,False,False,True
2,True,True,False,True,False,False,True,True,True,True,True,True,True,False,False
3,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True
4,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True
19996,True,False,True,True,False,True,False,True,True,True,False,True,True,True,True
19997,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True
19998,True,True,False,True,True,True,True,True,True,False,False,True,True,True,True


In [ ]:
## test data rmse 결과 확인하기. tokenizing이나 normalizing에서 생긴 오류가 있는건지 확인하기.

In [ ]:
path = '/content/drive/MyDrive/Lab/TabCSDI_plus/data_census_ft/MCAR-MCAR_seed-2.pk'
with open(path, "rb") as f:
  observed_values, observed_masks, gt_masks, cont_cols = pickle.load(f)

In [ ]:
path = '/content/drive/MyDrive/Lab/TabCSDI_plus/data_census_ft/MCAR-MCAR_seed-2_max-min_norm.pk'
with open(path, "rb") as f:
  observed_values_n, observed_masks_n, gt_masks_n = pickle.load(f)

In [ ]:
norm_obs = pd.DataFrame(observed_values_n)
norm_obs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.310811,0.039726,0.0000,0.00000,0.00023,0.404040,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1,0.459459,0.043681,0.8125,0.00000,0.00023,0.131313,2.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0
2,0.297297,0.133987,0.0000,0.00001,0.00000,0.000000,3.0,2.0,3.0,3.0,1.0,1.0,1.0,0.0,0.0
3,0.500000,0.147004,0.4375,0.00001,0.00023,0.404040,3.0,3.0,2.0,3.0,2.0,2.0,0.0,0.0,1.0
4,0.162162,0.217761,0.8125,0.00001,0.00023,0.404040,3.0,1.0,2.0,4.0,0.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.270270,0.047987,0.5625,0.00001,0.00023,0.808081,2.0,2.0,0.0,7.0,2.0,1.0,1.0,1.0,2.0
19996,0.243243,-0.000000,0.5000,0.00001,0.00000,0.404040,0.0,16.0,2.0,5.0,0.0,1.0,1.0,24.0,1.0
19997,0.756757,0.170301,0.2500,0.00001,0.00023,1.000000,0.0,0.0,0.0,5.0,1.0,1.0,2.0,1.0,1.0
19998,0.513514,0.221059,0.0000,0.00001,0.00023,0.444444,3.0,6.0,2.0,0.0,0.0,1.0,1.0,1.0,2.0


In [ ]:
pd.DataFrame(observed_values)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39.0,77516.0,0.0,0.0,0.0,40.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1,50.0,83311.0,13.0,0.0,0.0,13.0,2.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0
2,38.0,215646.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,0.0,0.0
3,53.0,234721.0,7.0,0.0,0.0,40.0,3.0,3.0,2.0,3.0,2.0,2.0,0.0,0.0,1.0
4,28.0,338409.0,13.0,0.0,0.0,40.0,3.0,1.0,2.0,4.0,0.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,36.0,89622.0,9.0,0.0,0.0,80.0,2.0,2.0,0.0,7.0,2.0,1.0,1.0,1.0,2.0
19996,34.0,0.0,8.0,0.0,0.0,40.0,0.0,16.0,2.0,5.0,0.0,1.0,1.0,24.0,1.0
19997,72.0,268861.0,4.0,0.0,0.0,99.0,0.0,0.0,0.0,5.0,1.0,1.0,2.0,1.0,1.0
19998,54.0,343242.0,0.0,0.0,0.0,44.0,3.0,6.0,2.0,0.0,0.0,1.0,1.0,1.0,2.0


In [ ]:
target

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.054054,0.176333,0.0625,0.00001,0.00023,0.323232,3.0,15.0,1.0,5.0,4.0,1.0,1.0,1.0,1.0
1,0.094595,0.113452,0.8125,0.00000,0.00023,0.404040,3.0,1.0,7.0,4.0,4.0,1.0,2.0,1.0,1.0
2,0.486487,0.092898,0.5625,0.00001,0.00023,0.484848,3.0,2.0,2.0,2.0,2.0,5.0,1.0,24.0,1.0
3,0.121622,0.197108,0.8125,0.00001,0.00023,0.404040,3.0,1.0,1.0,7.0,1.0,5.0,1.0,5.0,1.0
4,0.000000,0.155764,0.9375,0.00000,0.00000,0.000000,3.0,1.0,2.0,10.0,4.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.081081,0.308550,0.0000,0.00001,0.00023,0.080808,3.0,1.0,7.0,11.0,4.0,1.0,1.0,1.0,1.0
3996,0.351351,0.204480,0.0000,0.00001,0.00023,0.404040,3.0,1.0,3.0,10.0,1.0,1.0,2.0,1.0,1.0
3997,0.418919,0.122667,0.8125,0.00000,0.00023,0.606061,3.0,1.0,2.0,10.0,2.0,1.0,2.0,1.0,2.0
3998,0.500000,0.067009,0.3125,0.00001,0.00000,0.363636,3.0,5.0,3.0,10.0,1.0,1.0,2.0,10.0,1.0


In [ ]:
results = []
for i in range(50):
    # sample에서 i번째 부분 텐서를 선택
    part_tensor = a[:, i, :, :]

    # recover 함수 적용
    recovered = model.tokenizer.recover(part_tensor.values, len(cont_list))

    # 결과를 리스트에 추가
    results.append(recovered)

# 결과 텐서들을 합쳐서 최종 텐서 생성
final_tensor = torch.stack(results, dim=1)  # 결과의 shape: [4000, 50, 15]

NameError: name 'model' is not defined

## 진짜 결과 확인

In [ ]:
import os

### FT. MCAR MCAR

In [ ]:
from src.main_model_table_ft import TabCSDI
from src.utils_table import train, evaluate_ft, evaluate_ft_all
from dataset_census.dataset_census_ft2 import get_dataloader

path_model = "census_ft_fold5/MCARMCAR_20240316_134433"
path_1 = 'census_ft_fold5/MCARMCAR_20240316_134433'
path_2 = 'census_ft_fold5/MCARMCAR_20240316_134433'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed = open_results_ft(path_model, path_1, path_2, tmr, p_obs, mecha)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


In [ ]:
with open("./save/census_ft_fold5/MCARMCAR_20240316_134433/result_nsample50.pk", 'rb') as f :
    result_ft_mcarmcar = CPU_Unpickler(f).load()
with open("./save/census_ft_fold5/MCARMCAR_20240316_134433/full_result_nsample50.pk", 'rb') as f :
    fresult_ft_mcarmcar = CPU_Unpickler(f).load()

In [ ]:
ft_cont_list = [0, 1, 2, 3, 4, 5]
evalpoints = evalpoints.iloc[:,ft_cont_list]
diff = (target-imputed).iloc[:,ft_cont_list]
diff = diff.where(evalpoints==1)
rmse = np.sqrt((diff ** 2).mean())
rmse_avg = result_ft_mcarmcar[0]
error = result_ft_mcarmcar[1]
error_avg = error.mean()

ft_cont_list = [0, 1, 2, 3, 4, 5]
fevalpoints = fevalpoints.iloc[:,ft_cont_list]
fdiff = (ftarget-fimputed).iloc[:,ft_cont_list]
fdiff = fdiff.where(fevalpoints==1)
frmse = np.sqrt((fdiff ** 2).mean())
frmse_avg = frmse.mean()
ferror = fresult_ft_mcarmcar[1]
ferror_avg = ferror.mean()

In [ ]:
print("FT, MCAR-MCAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) FT, MCAR-MCAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

FT, MCAR-MCAR
RMSE:  0    161.098511
1     65.289787
2    116.006287
3     51.453136
4    116.403755
5    181.635773
dtype: float32
RMSE-avg:  115.31454467773438
Error:  [0.96296296 0.96171516 0.80837359 0.93343419 0.8449848  0.91693291
 0.3480916  0.99841772 0.2324159 ]
Error-avg:  0.7785920935394217


(for Full data) FT, MCAR-MCAR
RMSE:  0    164.663986
1     58.407818
2    107.348351
3     41.746632
4    104.872513
5    184.065170
dtype: float32
RMSE-avg:  110.18408
Error:  [0.98717949 0.9840569  0.99727183 0.71098846 0.46165566 0.95978621
 0.97715736 0.99951267 0.98937485]
Error-avg:  0.8963314912468774


### FT MAR MCAR

In [ ]:
# FT, MAR, MCAR
path_model = "census_ft_fold5/MARMCAR_20240317_041523"
path_1 = 'census_ft_fold5/MARMCAR_20240317_115217'
path_2 = 'census_ft_fold5/MARMCAR_20240317_115217'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed = open_results_ft(path_model, path_1, path_2, tmr, p_obs, mecha)
with open("./save/census_ft_fold5/MARMCAR_20240317_115217/result_nsample50.pk", 'rb') as f :
    result_ft_marmcar = CPU_Unpickler(f).load()
with open("./save/census_ft_fold5/MARMCAR_20240317_115217/full_result_nsample50.pk", 'rb') as f :
    fresult_ft_marmcar = CPU_Unpickler(f).load()

ft_cont_list = [0, 1, 2, 3, 4, 5]
evalpoints = evalpoints.iloc[:,ft_cont_list]
diff = (target-imputed).iloc[:,ft_cont_list]
diff = diff.where(evalpoints==1)
rmse = np.sqrt((diff ** 2).mean())
rmse_avg = result_ft_marmcar[0]
error = result_ft_marmcar[1]
error_avg = error.mean()

ft_cont_list = [0, 1, 2, 3, 4, 5]
fevalpoints = fevalpoints.iloc[:,ft_cont_list]
fdiff = (ftarget-fimputed).iloc[:,ft_cont_list]
fdiff = fdiff.where(fevalpoints==1)
frmse = np.sqrt((fdiff ** 2).mean())
frmse_avg = np.nanmean(frmse)
ferror = fresult_ft_marmcar[1]
ferror_avg = np.nanmean(ferror)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 1, 2, 3, 4, 5] [8, 16, 8, 14, 6, 5, 2, 41, 2]


In [ ]:
print("FT, MAR-MCAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) FT, MAR-MCAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

FT, MAR-MCAR
RMSE:  0     342.521790
1     185.617188
2     191.895676
3     159.924286
4    1279.185669
5    1052.082642
dtype: float32
RMSE-avg:  535.2045288085938
Error:  [0.99404762 0.95508982 0.96078431 0.89423077 0.66362253 0.59404389
 0.50390016 0.99837925 0.3196347 ]
Error-avg:  0.764859227756262


(for Full data) FT, MAR-MCAR
RMSE:  0     343.057434
1            NaN
2     163.765732
3     113.421753
4    1237.099976
5     999.749939
dtype: float32
RMSE-avg:  571.419
Error:  [       nan 0.9994985         nan 0.94699211 0.92806285 0.99949887
 0.52245805 1.         0.87394744]
Error-avg:  0.8957796874738683


### Analog MCAR MCAR

In [ ]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path_2 = 'census_analog_fold5/MCARMCAR_20240319_115432'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed = open_results_an(path_model, path_1, path_2, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MCARMCAR_20240319_115432/result_nsample50.pk", 'rb') as f :
    result_an_mcarmcar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MCARMCAR_20240319_115432/full_result_nsample50.pk", 'rb') as f :
    fresult_an_mcarmcar = CPU_Unpickler(f).load()

an_cont_list = [0, 5, 11, 28, 29, 30]
evalpoints_cont = evalpoints.iloc[:,an_cont_list]
diff = (target-imputed).iloc[:,an_cont_list]
diff = diff.where(evalpoints_cont==1)
rmse = np.sqrt((diff ** 2).mean())
rmse_avg = result_an_mcarmcar[0]
# error = result_an_mcarmcar[1]
# error_avg = error.mean()

with open("./data_census_analog/transform.pk", "rb") as f:
    _, cont_cols, saved_cat_dict = pickle.load(f)

err_total = np.zeros([len(saved_cat_dict)])
err_total_eval_nums = np.zeros([len(saved_cat_dict)])
for i in range(len(saved_cat_dict)):
    cate_cols = saved_cat_dict[str(i)]
    matched_nums = (
        (
            (
                samples.values[:, cate_cols]
                == target.values[:, cate_cols]
            )
            * evalpoints.values[:, cate_cols]
        )
        .all(1)
        .sum()
    )
    eval_nums = evalpoints.values[:, cate_cols].sum() / len(
        cate_cols
    )
    err_total[i] += eval_nums - matched_nums
    err_total_eval_nums[i] += eval_nums

error = err_total / err_total_eval_nums
error_avg = error.mean()

#############
an_cont_list = [0, 5, 11, 28, 29, 30]
fevalpoints_cont = fevalpoints.iloc[:,an_cont_list]
fdiff = (ftarget-fimputed).iloc[:,an_cont_list]
fdiff = fdiff.where(fevalpoints_cont==1)
frmse = np.sqrt((fdiff ** 2).mean())
frmse_avg = fresult_an_mcarmcar[0]

with open("./data_census_analog/transform.pk", "rb") as f:
    _, cont_cols, saved_cat_dict = pickle.load(f)

ferr_total = np.zeros([len(saved_cat_dict)])
ferr_total_eval_nums = np.zeros([len(saved_cat_dict)])
for i in range(len(saved_cat_dict)):
    cate_cols = saved_cat_dict[str(i)]
    fmatched_nums = (
        (
            (
                fsamples.values[:, cate_cols]
                == ftarget.values[:, cate_cols]
            )
            * fevalpoints.values[:, cate_cols]
        )
        .all(1)
        .sum()
    )
    feval_nums = fevalpoints.values[:, cate_cols].sum() / len(
        cate_cols
    )
    ferr_total[i] += feval_nums - fmatched_nums
    ferr_total_eval_nums[i] += feval_nums

ferror = ferr_total / ferr_total_eval_nums
ferror_avg = ferror.mean()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


In [ ]:
print("Analog, MCAR-MCAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) Analog, MCAR-MCAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

Analog, MCAR-MCAR
RMSE:  0     0.884018
5     1.000341
11    0.764632
28    1.010497
29    1.864238
30    2.448273
dtype: float32
RMSE-avg:  1.3297713994979858
Error:  [1.         0.959375   0.9306184  1.         0.77794793 0.84057971
 0.60816944 0.99683544 0.97706422]
Error-avg:  0.898954460825974


(for Full data) Analog, MCAR-MCAR
RMSE:  0     0.381689
5     0.698798
11    0.294508
28    0.627329
29    0.936683
30    1.061144
dtype: float32
RMSE-avg:  0.7280893325805664
Error:  [0.00099083 0.7640621  0.88875124 0.00326797 0.82781457 0.97395833
 0.84696438 0.41569201 0.08574253]
Error-avg:  0.5341382179009996


In [ ]:
with open("./save/census_analog_fold5/MCARMCAR_20240320_112245/full_real_data.pk", 'rb') as f :
    real = pickle.load(f)

In [ ]:
pd.DataFrame(np.array(real[0].squeeze().cpu()))

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.108108,-1.0,1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.270270,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00023,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,0.308550,-1.0,1.0,-1.0,1.0,...,0.00023,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,0.204480,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,1.0,-1.0,1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
19998,0.500000,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,-1.0,1.0,1.0,...,0.00023,0.363636,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


In [ ]:
import math
cate_columns = [i for i in range(39) if i not in an_cont_list]

# math.copysign(1, x)를 벡터화하여 적용하는 함수 정의
vectorized_copysign = np.vectorize(lambda x: int(math.copysign(1, x)))

# 데이터프레임의 모든 값을 copysign(1, x)의 결과로 치환
fimputed.iloc[:,cate_columns] = fimputed.iloc[:,cate_columns].apply(vectorized_copysign)

fimputed

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,-1.0,-1.0,-1.0,-0.000000,-1.0,-1.0,-1.0,-1.0,...,0.00023,-0.000000,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
1,0.108108,-1.0,-1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,0.000000,-1.0,-1.0,-1.0,-1.0,-0.000000,-1.0,-1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,-0.000000,-1.0,1.0,-1.0,-1.0,0.062663,1.0,1.0,1.0,-1.0,...,0.00000,0.484848,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,-0.000000,1.0,-1.0,-1.0,-1.0,...,0.00023,-0.000000,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.000000,-1.0,-1.0,-1.0,-1.0,-0.779809,-1.0,-1.0,1.0,-1.0,...,-0.00000,-0.000000,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
19996,0.000000,-1.0,-1.0,-1.0,-1.0,-0.597566,-1.0,1.0,-1.0,1.0,...,-0.00000,-0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
19997,-0.000000,-1.0,-1.0,-1.0,-1.0,-0.000000,-1.0,1.0,-1.0,-1.0,...,-0.00000,-0.787804,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
19998,0.000000,-1.0,-1.0,-1.0,-1.0,-0.000000,1.0,1.0,-1.0,-1.0,...,-0.00000,-0.000000,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
ftarget

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
1,0.108108,-1.0,-1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,-0.000000,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00000,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,-0.000000,-1.0,1.0,-1.0,1.0,...,0.00023,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,-0.000000,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.000000,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
19998,0.500000,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,-1.0,1.0,1.0,...,0.00023,0.363636,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


### Analog MAR MCAR

In [ ]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

path_model = "census_analog_fold5/MARMCAR_20240320_123318"
path_1 = 'census_analog_fold5/MARMCAR_20240320_123318'
path_2 = 'census_analog_fold5/MARMCAR_20240320_123318'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed = open_results_an(path_model, path_1, path_2, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MARMCAR_20240320_123318/result_nsample50.pk", 'rb') as f :
    result_an_marmcar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MARMCAR_20240320_123318/full_result_nsample50.pk", 'rb') as f :
    fresult_an_marmcar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results(evalpoints, target, imputed, result_an_marmcar, fevalpoints, ftarget, fimputed, fresult_an_marmcar)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


<ipython-input-11-c781702eee2b>:66: RuntimeWarning: divide by zero encountered in divide
  ferror = ferr_total / ferr_total_eval_nums


In [ ]:
ferror # 왜 0~1 아닌 값들이 나오냐고....

array([ 1.39938443,  1.30082823,  1.65667075,  2.63990998,  1.35217707,
       -0.25760266,        -inf,  1.6071343 ,        -inf])

In [ ]:
error

array([0.93386243, 0.96640827, 1.        , 0.99761905, 0.93617021,
       0.88627451, 0.57160648, 0.99745547, 0.68130489])

In [ ]:
fsamples

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,-714.737915,1.0,-1.0,1.0,1.0,0.000000,1.0,-1.0,1.0,1.0,...,-0.000000,-0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0
1,0.000000,1.0,-1.0,1.0,1.0,841.443237,-1.0,-1.0,-1.0,1.0,...,-0.000000,-4101.279785,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.000000,1.0,-1.0,1.0,1.0,329.363617,1.0,-1.0,1.0,1.0,...,-2.849268,-0.000000,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0
3,0.000000,1.0,-1.0,1.0,1.0,0.000000,1.0,1.0,-1.0,1.0,...,-0.000000,-0.000000,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
4,0.000000,1.0,-1.0,1.0,-1.0,-1321.166260,1.0,-1.0,1.0,1.0,...,-0.000000,-0.000000,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-498.456146,1.0,1.0,1.0,-1.0,0.000000,-1.0,-1.0,-1.0,1.0,...,-3.222399,-1188.181030,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
19996,874.291077,1.0,-1.0,1.0,1.0,0.000000,-1.0,-1.0,1.0,1.0,...,-1.258067,-0.000000,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0
19997,-6047.441406,1.0,-1.0,1.0,1.0,-228.978851,1.0,1.0,1.0,-1.0,...,-1.752022,-0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
19998,0.000000,1.0,-1.0,1.0,1.0,-487.525421,1.0,-1.0,-1.0,1.0,...,-0.000000,-0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0


In [ ]:
ftarget

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.108108,-1.0,1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.270270,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00023,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,0.308550,-1.0,1.0,-1.0,1.0,...,0.00023,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,0.204480,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,1.0,-1.0,1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
19998,0.500000,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,-1.0,1.0,1.0,...,0.00023,0.363636,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


In [ ]:
fimputed

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,1.0,-1.0,1.0,1.0,0.000000,-1.0,-1.0,-1.0,-1.0,...,0.000230,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.108108,-1.0,1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.000230,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,0.554054,1.0,-1.0,1.0,1.0,329.363617,-1.0,1.0,1.0,-1.0,...,0.000230,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.270270,-1.0,1.0,-1.0,-1.0,0.062663,1.0,1.0,-1.0,1.0,...,0.000230,0.484848,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,-1321.166260,1.0,-1.0,-1.0,-1.0,...,0.000230,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-498.456146,1.0,1.0,1.0,-1.0,0.000000,-1.0,-1.0,-1.0,1.0,...,-3.222399,-1188.181030,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
19996,874.291077,1.0,-1.0,1.0,1.0,0.000000,-1.0,-1.0,1.0,1.0,...,-1.258067,-0.000000,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0
19997,-6047.441406,1.0,-1.0,1.0,1.0,-228.978851,1.0,1.0,1.0,-1.0,...,-1.752022,-0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
19998,0.000000,1.0,-1.0,1.0,1.0,-487.525421,1.0,-1.0,-1.0,1.0,...,-0.000000,-0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0


### Analog MCAR MAR

In [ ]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

path_model = "census_analog_fold5/MCARMAR_20240321_080014"
path_1 = 'census_analog_fold5/MCARMAR_20240321_080014'
path_2 = 'census_analog_fold5/MCARMAR_20240321_105225'
tmr = 0.4
mecha = 'MAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed = open_results_an(path_model, path_1, path_2, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MCARMAR_20240321_080014/result_nsample50.pk", 'rb') as f :
    result_an_mcarmar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MCARMAR_20240321_105225/full_result_nsample50.pk", 'rb') as f :
    fresult_an_mcarmar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results(evalpoints, target, imputed, result_an_mcarmar, fevalpoints, ftarget, fimputed, fresult_an_mcarmar)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


<ipython-input-11-c781702eee2b>:36: RuntimeWarning: invalid value encountered in divide
  error = err_total / err_total_eval_nums


In [ ]:
frmse

0     14054.621094
5     18241.335938
11     5447.790527
28    11544.540039
29    24924.402344
30    35843.035156
dtype: float32

In [ ]:
ferror
# test 결과는 정상?인데 full에서 결과가 이상함.

array([ 1.23132736,  1.47226335,  1.41184173,  1.74433081,  1.91172596,
       -2.56183746,  3.34119418, -3.18466899,  2.10427477])

In [ ]:
# 결과 반환 값에 real 추가했음. 이거 반영하도록 result 가져오는 함수도 수정해야됨!!! 미친...
# 순서대로 generated_samples/real/target/evalpoint/observed_point/observed_time...
with open("./save/census_analog_fold5/MCARMAR_20240321_105225/full_generated_outputs_nsample50.pk", 'rb') as f :
    fresults = CPU_Unpickler(f).load()

In [ ]:
len(fresults)

8

In [ ]:
pd.DataFrame(fresults[1].squeeze().cpu().numpy())

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.108108,-1.0,1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.270270,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00023,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,0.308550,-1.0,1.0,-1.0,1.0,...,0.00023,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,0.204480,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,1.0,-1.0,1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
19998,0.500000,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,-1.0,1.0,1.0,...,0.00023,0.363636,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


In [ ]:
pd.DataFrame(fresults[2].squeeze().cpu().numpy())

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
1,0.108108,-1.0,-1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,-0.000000,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00000,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,-0.000000,-1.0,1.0,-1.0,1.0,...,0.00023,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,-0.000000,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.000000,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
19998,0.500000,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,-1.0,1.0,1.0,...,0.00023,0.363636,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


In [ ]:
pd.DataFrame(fresults[3].squeeze().cpu().numpy())
## 연속형을 죄다 0으로 채워버리는 경향이 좀 있는 듯 함...

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

path_model = "census_analog_fold5/MCARMAR_20240321_080014"
path_1 = 'census_analog_fold5/MCARMAR_20240321_080014'
path_2 = 'census_analog_fold5/MCARMAR_20240321_105225'
tmr = 0.4
mecha = 'MAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path_model, path_1, path_2, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MCARMAR_20240321_080014/result_nsample50.pk", 'rb') as f :
    result_an_mcarmar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MCARMAR_20240321_105225/full_result_nsample50.pk", 'rb') as f :
    fresult_an_mcarmar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_mcarmar, fevalpoints, ftarget, fimputed, fresult_an_mcarmar, freal)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


<ipython-input-36-1fddc63e38d8>:36: RuntimeWarning: invalid value encountered in divide
  error = err_total / err_total_eval_nums


In [ ]:
ferror

array([0.91330196, 0.99210995, 0.99950446, 0.92357969, 0.98920775,
       0.86731151, 1.        , 0.99707602, 0.82629108])

In [ ]:
frmse.mean()

0.5761247

### (real data에 대한 간단한 확인)

In [ ]:
with open("./save/census_analog_fold5/MCARMCAR_20240319_115432_/full_real_data.pk", 'rb') as f :
    real1 = CPU_Unpickler(f).load()

In [ ]:
real1 = pd.DataFrame(real1[0].squeeze().cpu())

In [ ]:
(real1 == freal).sum()

0     20000
1     20000
2     20000
3     20000
4     20000
5     20000
6     20000
7     20000
8     20000
9     20000
10    20000
11    20000
12    20000
13    20000
14    20000
15    20000
16    20000
17    20000
18    20000
19    20000
20    20000
21    20000
22    20000
23    20000
24    20000
25    20000
26    20000
27    20000
28    20000
29    20000
30    20000
31    20000
32    20000
33    20000
34    20000
35    20000
36    20000
37    20000
38    20000
dtype: int64

### 결과확인 다시하기

### Analog MCAR MCAR

In [41]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MCARMCAR_20240319_115432_'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MCARMCAR_20240319_115432_/result_nsample50.pk", 'rb') as f :
    result_an_mcarmcar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MCARMCAR_20240319_115432_/full_result_nsample50.pk", 'rb') as f :
    fresult_an_mcarmcar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_mcarmcar, fevalpoints, ftarget, fimputed, fresult_an_mcarmcar, freal)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


In [42]:
print("Analog, MCAR-MCAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) Analog, MCAR-MCAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

Analog, MCAR-MCAR
RMSE:  0     0.884018
5     1.000341
11    0.764632
28    1.010497
29    1.864238
30    2.448273
dtype: float32
RMSE-avg:  1.3297713994979858
Error:  [1.         0.959375   0.9306184  1.         0.77794793 0.84057971
 0.60816944 0.99683544 0.97706422]
Error-avg:  0.898954460825974


(for Full data) Analog, MCAR-MCAR
RMSE:  0     0.737718
5     0.886088
11    0.706366
28    0.706221
29    1.054481
30    1.510125
dtype: float32
RMSE-avg:  0.9335
Error:  [1.         0.94884194 0.94375619 0.99974862 0.76894776 0.84375
 0.64149523 0.99878168 0.97380776]
Error-avg:  0.9021254644920939


### Analog MAR MCAR

In [43]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MARMCAR_20240320_123318'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MARMCAR_20240320_123318/result_nsample50.pk", 'rb') as f :
    result_an_marmcar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MARMCAR_20240320_123318/full_result_nsample50.pk", 'rb') as f :
    fresult_an_marmcar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_marmcar, fevalpoints, ftarget, fimputed, fresult_an_marmcar, freal)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


<ipython-input-14-170c6b38e441>:67: RuntimeWarning: invalid value encountered in divide
  ferror = ferr_total / ferr_total_eval_nums


In [44]:
print("Analog, MAR-MCAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) Analog, MAR-MCAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

Analog, MAR-MCAR
RMSE:  0     0.892529
5     0.577688
11    0.453978
28    0.822517
29    0.952266
30    1.233923
dtype: float32
RMSE-avg:  0.8120233416557312
Error:  [0.93386243 0.96640827 1.         0.99761905 0.93617021 0.88627451
 0.57160648 0.99745547 0.68130489]
Error-avg:  0.8856334791765836


(for Full data) Analog, MAR-MCAR
RMSE:  0     0.955160
5     0.613688
11         NaN
28    0.733435
29    0.876794
30    1.174617
dtype: float32
RMSE-avg:  0.8707388
Error:  [       nan 0.96757028 0.99909228        nan 0.92762231        nan
        nan 0.99868421        nan]
Error-avg:  nan


### Analog MCAR MAR

In [45]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MCARMAR_20240321_080014_'
tmr = 0.4
mecha = 'MAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MCARMAR_20240321_080014_/result_nsample50.pk", 'rb') as f :
    result_an_mcarmar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MCARMAR_20240321_080014_/full_result_nsample50.pk", 'rb') as f :
    fresult_an_mcarmar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_mcarmar, fevalpoints, ftarget, fimputed, fresult_an_mcarmar, freal)

[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
<ipython-input-14-170c6b38e441>:36: RuntimeWarning: invalid value encountered in divide
  error = err_total / err_total_eval_nums


In [46]:
print("Analog, MCAR-MAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) Analog, MCAR-MAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

Analog, MCAR-MAR
RMSE:  0     0.379101
5          NaN
11         NaN
28         NaN
29    0.586814
30    1.207028
dtype: float32
RMSE-avg:  nan
Error:  [0.9025894         nan 0.99938688        nan        nan 0.84944238
 1.         0.99813317 0.82009627]
Error-avg:  nan


(for Full data) Analog, MCAR-MAR
RMSE:  0     0.392456
5     0.390943
11    0.412805
28    0.588025
29    0.602634
30    1.213895
dtype: float32
RMSE-avg:  0.60012627
Error:  [0.90116423 0.99058285 0.99900892 0.90573152 0.98700025 0.85119048
 1.         0.99683236 0.79960465]
Error-avg:  0.9367905827814482


### Analog MAR MAR

In [47]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MARMAR_20240321_162138'
tmr = 0.4
mecha = 'MAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MARMAR_20240321_162138/result_nsample50.pk", 'rb') as f :
    result_an_marmar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MARMAR_20240321_162138/full_result_nsample50.pk", 'rb') as f :
    fresult_an_marmar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_marmar, fevalpoints, ftarget, fimputed, fresult_an_marmar, freal)

[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
<ipython-input-14-170c6b38e441>:36: RuntimeWarning: invalid value encountered in divide
  error = err_total / err_total_eval_nums
<ipython-input-14-170c6b38e441>:67: RuntimeWarning: invalid value encountered in divide
  ferror = ferr_total / ferr_total_eval_nums


In [48]:
print("Analog, MAR-MAR")
print('RMSE: ', rmse)
print('RMSE-avg: ', rmse_avg)
print('Error: ', error)
print('Error-avg: ', error_avg)
print('\n')
print("(for Full data) Analog, MAR-MAR")
print('RMSE: ', frmse)
print('RMSE-avg: ', frmse_avg)
print('Error: ', ferror)
print('Error-avg: ', ferror_avg)

Analog, MAR-MAR
RMSE:  0     0.413340
5          NaN
11         NaN
28         NaN
29    0.384782
30    0.996405
dtype: float32
RMSE-avg:  nan
Error:  [0.89481409        nan 0.99442897        nan        nan 0.77683897
 1.         0.99587629 0.90904548]
Error-avg:  nan


(for Full data) Analog, MAR-MAR
RMSE:  0     0.439925
5     0.448458
11         NaN
28    0.681777
29    0.391816
30    1.015504
dtype: float32
RMSE-avg:  0.59549606
Error:  [       nan 0.99457831 0.99283913        nan 0.98711496        nan
        nan 0.99736842        nan]
Error-avg:  nan


### 기타

In [25]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MARMCAR_20240320_123318'
tmr = 0.2
mecha = 'MCAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MARMCAR_20240320_123318/result_nsample50.pk", 'rb') as f :
    result_an_marmcar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MARMCAR_20240320_123318/full_result_nsample50.pk", 'rb') as f :
    fresult_an_marmcar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_marmcar, fevalpoints, ftarget, fimputed, fresult_an_marmcar, freal)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


<ipython-input-14-170c6b38e441>:67: RuntimeWarning: invalid value encountered in divide
  ferror = ferr_total / ferr_total_eval_nums


In [26]:
fevalpoints

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
19998,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
fimputed

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,-0.458531,-1.0,1.0,-1.0,-1.0,...,-0.817690,-0.614088,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,-0.523583,-1.0,1.0,-1.0,-1.0,0.146360,1.0,1.0,1.0,-1.0,...,-0.803019,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,-0.627827,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.000230,-0.599371,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,-0.795891,-1.0,1.0,-1.0,-1.0,-0.338719,-1.0,1.0,-1.0,1.0,...,-1.069310,-0.724433,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0
4,-0.697520,-1.0,1.0,-1.0,-1.0,0.117097,-1.0,1.0,-1.0,-1.0,...,-0.754399,-0.915212,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,-0.441395,1.0,1.0,1.0,-1.0,...,0.000230,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,-0.450847,1.0,1.0,-1.0,-1.0,...,0.000230,-0.807481,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,1.0,-1.0,1.0,...,0.000230,-0.774334,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
19998,-0.615438,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,1.0,1.0,-1.0,...,-1.005637,-0.562845,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


In [28]:
freal

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.108108,-1.0,1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.270270,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00023,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.081081,-1.0,-1.0,-1.0,1.0,0.308550,-1.0,1.0,-1.0,1.0,...,0.00023,0.080808,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19996,0.351351,-1.0,1.0,-1.0,-1.0,0.204480,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
19997,0.418919,-1.0,1.0,-1.0,-1.0,0.122667,-1.0,1.0,-1.0,1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
19998,0.500000,-1.0,1.0,-1.0,-1.0,0.067009,-1.0,-1.0,1.0,1.0,...,0.00023,0.363636,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0


In [29]:
ferror

array([       nan, 0.96757028, 0.99909228,        nan, 0.92762231,
              nan,        nan, 0.99868421,        nan])

In [30]:
with open("./data_census_analog/transform.pk", "rb") as f:
  _, cont_cols, saved_cat_dict = pickle.load(f)

ferr_total = np.zeros([len(saved_cat_dict)])
ferr_total_eval_nums = np.zeros([len(saved_cat_dict)])
for i in range(len(saved_cat_dict)):
    cate_cols = saved_cat_dict[str(i)]
    fmatched_nums = (
        (
            (
                fimputed.values[:, cate_cols] ###
                == freal.values[:, cate_cols]
            )
            * fevalpoints.values[:, cate_cols]
        )
        .all(1)
        .sum()
    )
    feval_nums = fevalpoints.values[:, cate_cols].sum() / len(
        cate_cols
    )
    ferr_total[i] += feval_nums - fmatched_nums
    ferr_total_eval_nums[i] += feval_nums

ferror = ferr_total / ferr_total_eval_nums
ferror_avg = ferror.mean()

<ipython-input-30-1a3a824e427c>:25: RuntimeWarning: invalid value encountered in divide
  ferror = ferr_total / ferr_total_eval_nums


In [31]:
cate_cols = saved_cat_dict['1'] # [6,7,8,9,10]
fmatched_nums = (
        (
            (
                fimputed.values[:, cate_cols] ###
                == freal.values[:, cate_cols]
            )
            * fevalpoints.values[:, cate_cols]
        )
        .all(1)
        .sum()
    )
fmatched_nums

323

In [32]:
((fimputed.values[:, cate_cols] == freal.values[:, cate_cols]) * fevalpoints.values[:, cate_cols]).all(1).sum()

323

In [33]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MCARMAR_20240321_080014_'
tmr = 0.4
mecha = 'MAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MCARMAR_20240321_080014_/result_nsample50.pk", 'rb') as f :
    result_an_mcarmar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MCARMAR_20240321_080014_/full_result_nsample50.pk", 'rb') as f :
    fresult_an_mcarmar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_mcarmar, fevalpoints, ftarget, fimputed, fresult_an_mcarmar, freal)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


<ipython-input-14-170c6b38e441>:36: RuntimeWarning: invalid value encountered in divide
  error = err_total / err_total_eval_nums


In [34]:
ferror

array([0.90116423, 0.99058285, 0.99900892, 0.90573152, 0.98700025,
       0.85119048, 1.        , 0.99683236, 0.79960465])

In [35]:
from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_analog, evaluate_analog_all
from dataset_census.dataset_census_analog2 import get_dataloader

# path_model = "census_analog_fold5/MCARMCAR_20240319_092255"
# path_1 = 'census_analog_fold5/MCARMCAR_20240319_115432'
path = 'census_analog_fold5/MARMAR_20240321_162138'
tmr = 0.4
mecha = 'MAR'
p_obs = None

target, samples, evalpoints, imputed, ftarget, fsamples, fevalpoints, fimputed, freal = open_results_an2(path, path, path, tmr, p_obs, mecha)

with open("./save/census_analog_fold5/MARMAR_20240321_162138/result_nsample50.pk", 'rb') as f :
    result_an_marmar = CPU_Unpickler(f).load()
with open("./save/census_analog_fold5/MARMAR_20240321_162138/full_result_nsample50.pk", 'rb') as f :
    fresult_an_marmar = CPU_Unpickler(f).load()

rmse, rmse_avg, error, error_avg, frmse, frmse_avg, ferror, ferror_avg = analog_results2(evalpoints, target, imputed, result_an_marmar, fevalpoints, ftarget, fimputed, fresult_an_marmar, freal)

[0, 5, 11, 28, 29, 30] {'0': [1, 2, 3, 4], '1': [6, 7, 8, 9, 10], '2': [12, 13, 14, 15], '3': [16, 17, 18, 19], '4': [20, 21, 22], '5': [23, 24, 25], '6': [26, 27], '7': [31, 32, 33, 34, 35, 36], '8': [37, 38]}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
<ipython-input-14-170c6b38e441>:36: RuntimeWarning: invalid value encountered in divide
  error = err_total / err_total_eval_nums
<ipython-input-14-170c6b38e441>:67: RuntimeWarning: invalid value encountered in divide
  ferror = ferr_total / ferr_total_eval_nums


In [36]:
frmse

0     0.439925
5     0.448458
11         NaN
28    0.681777
29    0.391816
30    1.015504
dtype: float32

In [37]:
ferror

array([       nan, 0.99457831, 0.99283913,        nan, 0.98711496,
              nan,        nan, 0.99736842,        nan])

In [50]:
freal[:10]

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,0.007455,-1.0,-1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.108108,-1.0,1.0,-1.0,-1.0,0.146360,1.0,-1.0,-1.0,-1.0,...,0.00023,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,0.554054,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.00023,0.363636,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.270270,-1.0,1.0,-1.0,-1.0,0.062663,-1.0,1.0,-1.0,1.0,...,0.00023,0.484848,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0
4,0.256757,-1.0,1.0,-1.0,-1.0,0.117097,1.0,-1.0,-1.0,-1.0,...,0.00023,0.606061,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
5,0.162162,-1.0,1.0,-1.0,-1.0,0.038780,-1.0,1.0,1.0,-1.0,...,0.00023,0.404040,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0
6,0.175676,-1.0,1.0,-1.0,-1.0,0.160033,-1.0,1.0,-1.0,1.0,...,0.00023,0.454545,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
7,0.202703,-1.0,1.0,-1.0,-1.0,0.147460,-1.0,-1.0,1.0,1.0,...,0.00023,0.303030,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
8,0.851351,-1.0,1.0,-1.0,-1.0,0.104353,-1.0,-1.0,1.0,1.0,...,0.00023,0.303030,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
9,0.162162,-1.0,1.0,-1.0,-1.0,0.080520,-1.0,1.0,-1.0,-1.0,...,0.00023,0.404040,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0


In [51]:
fimputed[:10]

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.189189,-1.0,1.0,-1.0,-1.0,-0.155004,1.0,1.0,1.0,1.0,...,0.092578,-0.600979,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
1,0.204019,-1.0,1.0,-1.0,-1.0,0.146360,1.0,1.0,1.0,1.0,...,-0.156901,0.505050,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0
2,-0.047268,-1.0,1.0,-1.0,-1.0,0.135516,-1.0,1.0,1.0,-1.0,...,0.000230,-0.470108,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
3,0.094597,-1.0,1.0,-1.0,-1.0,0.495919,-1.0,1.0,-1.0,1.0,...,-0.009403,-0.685813,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0
4,0.153159,-1.0,1.0,-1.0,-1.0,0.117097,1.0,1.0,1.0,1.0,...,0.533242,-0.724188,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
5,-0.034017,-1.0,1.0,-1.0,-1.0,0.199187,1.0,1.0,1.0,1.0,...,0.000230,0.404040,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0
6,-0.040520,-1.0,1.0,-1.0,-1.0,-0.046624,-1.0,1.0,-1.0,1.0,...,0.691692,-0.363368,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0
7,0.202703,-1.0,1.0,-1.0,-1.0,0.147460,1.0,1.0,1.0,1.0,...,0.245479,-0.715093,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0
8,0.851351,-1.0,1.0,-1.0,-1.0,0.104353,1.0,1.0,-1.0,1.0,...,0.609812,0.303030,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
9,0.162162,-1.0,1.0,-1.0,-1.0,-0.536911,1.0,1.0,1.0,1.0,...,0.000230,-0.648597,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0


In [49]:
fsamples

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.000000,-0.0,0.0,-0.0,-0.0,-0.155004,1.0,1.0,1.0,1.0,...,0.092578,-0.600979,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0
1,0.204019,-0.0,0.0,-0.0,-0.0,0.000000,1.0,1.0,1.0,1.0,...,-0.156901,-0.000000,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0
2,-0.047268,-0.0,0.0,-0.0,-0.0,0.000000,-0.0,0.0,0.0,-0.0,...,0.000000,-0.470108,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0
3,0.094597,-0.0,0.0,-0.0,-0.0,0.495919,-0.0,0.0,0.0,0.0,...,-0.009403,-0.685813,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-0.0,-0.0
4,0.153159,-0.0,-0.0,-0.0,0.0,-0.000000,1.0,1.0,1.0,1.0,...,0.533242,-0.724188,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.000000,-0.0,-0.0,-0.0,0.0,-0.549646,1.0,1.0,1.0,1.0,...,0.000000,-0.000000,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0
19996,0.000000,-0.0,-0.0,0.0,-0.0,0.267265,1.0,1.0,1.0,1.0,...,0.000000,-0.649383,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-0.0
19997,-0.000000,0.0,-0.0,-0.0,-0.0,0.000000,-0.0,0.0,-0.0,-0.0,...,0.000000,-0.701493,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-0.0,0.0
19998,0.034411,-0.0,-0.0,-0.0,-0.0,0.000000,1.0,1.0,1.0,1.0,...,0.336576,-0.695588,-0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0


In [52]:
fevalpoints[:10]

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
